# Import Demand

In [532]:
map_file_name = 'data/disciplinas/dia_hora_instituto_ponto_linha_18h.json'
hora = 1080
def intervalo_horario(horario):
    return '17:30' <= horario <= '19:00'
def intervalo_horario_am_pm(horario):
    return '5:30:00 PM' <= horario <= '7:00:00 PM'

## Aulas MATRUSP

##### Read jsons

In [533]:
import os
import json

In [534]:
files = []
dir_path = 'data/disciplinas/json/'
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(path)

In [535]:
len(files)

5205

In [536]:
disciplinas = []
for file in files:
    with open(dir_path + file, 'r+') as f:
        content = json.loads(f.read())
        disciplinas.append(content)

In [537]:
disciplinas[1000]

{'unidade': 'Escola Politécnica',
 'departamento': 'Eng Telecomunicações e Controle',
 'campus': 'São Paulo',
 'codigo': 'PTC3502',
 'nome': 'Estágio Supervisionado',
 'creditos_aula': 1,
 'creditos_trabalho': 6,
 'objetivos': 'Realização de um estágio relacionado ao futuro exercício profissional, com a supervisão de um professor do curso.',
 'programa_resumido': 'Estágio em empresa, universidade ou instituição de pesquisa.',
 'turmas': [{'codigo': '2022250',
   'inicio': '15/08/2022',
   'fim': '21/12/2022',
   'tipo': 'Teórica',
   'horario': [{'dia': 'sab',
     'inicio': '09:20',
     'fim': '10:10',
     'professores': ['Cristiano Magalhaes Panazio']}],
   'vagas': {'Obrigatória': {'vagas': 35,
     'inscritos': 8,
     'pendentes': 0,
     'matriculados': 8,
     'grupos': {'EP - Engenharia Elétrica - EC3 (Telecomunicações)': {'vagas': 35,
       'inscritos': 6,
       'pendentes': 0,
       'matriculados': 6}}},
    'Extracurricular': {'vagas': 2,
     'inscritos': 0,
     'pend

##### Filtrar Institutos CUASO

In [538]:
fora_cuaso = ['Escola de Artes, Ciências e Humanidades', 'Escola de Enfermagem', 'Faculdade de Direito', 'Faculdade de Medicina',
              'Faculdade de Saúde Pública', 'Faculdade de Saúde Pública e Faculdade de Ciências Farmacêuticas', 'Escola de Enfermagem de Ribeirão Preto',
              'Museu de Zoologia']
def is_disciplina_cuaso(disciplina, fora_cuaso):
    if disciplina['campus'] != 'São Paulo':
        return False
    if disciplina['unidade'] in fora_cuaso:
        return False
    return True

In [539]:
disciplinas_cuaso = []
for disciplina in disciplinas:
    if is_disciplina_cuaso(disciplina, fora_cuaso):
        disciplinas_cuaso.append(disciplina)
len(disciplinas_cuaso)

2352

In [540]:
unidades_matrusp = {}
for disciplina in disciplinas_cuaso:
    unidades_matrusp[disciplina['unidade']] = 1
unidades_matrusp

{'Instituto Oceanográfico': 1,
 'Escola Politécnica': 1,
 'Escola de Comunicações e Artes': 1,
 'Instituto de Matemática e Estatística': 1,
 'Faculdade de Filosofia, Letras e Ciências Humanas': 1,
 'Instituto de Química': 1,
 'Instituto de Ciências Biomédicas': 1,
 'Instituto de Geociências': 1,
 'Instituto de Biociências': 1,
 'Instituto de Psicologia': 1,
 'Faculdade de Medicina Veterinária e Zootecnia': 1,
 'Faculdade de Ciências Farmacêuticas': 1,
 'Escola de Educação Física e Esporte': 1,
 'Faculdade de Educação': 1,
 'Instituto de Física': 1,
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 1,
 'Faculdade de Odontologia': 1,
 'Faculdade de Arquitetura e Urbanismo': 1,
 'Instituto de Relações Internacionais': 1,
 'Faculdade de Economia, Administração, Contabilidade e Atuária': 1,
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 1,
 'Instituto de Estudos Brasileiros': 1,
 'Museu de Arqueologia e Etnologia': 1

##### Gerar Lista de Aulas

In [541]:
aulas = []
for i, disciplina in enumerate(disciplinas_cuaso):
    for turma in disciplina['turmas']:
        total_matriculados = 0
        for k, vaga in turma['vagas'].items():
            total_matriculados += vaga['matriculados']
            
        if turma['horario'] != None:
            for horarios in turma['horario']:
                aula = {}
                aula['indice_disc'] = i
                aula['codigo'] = disciplina['codigo']
                aula['codigo_turma'] = turma['codigo']
                aula['unidade'] = disciplina['unidade']
                aula['dia'] = horarios['dia']
                aula['horario_inicio'] = horarios['inicio']
                aula['total_matriculados'] = total_matriculados
                aulas.append(aula)

##### Filtrar Aulas por Intervalo Horário

In [542]:
aulas_intervalo_horario = []
for aula in aulas:
    if intervalo_horario(aula['horario_inicio']):
        aulas_intervalo_horario.append(aula)

# Pesquisa

##### Ler Pesquisa

In [543]:
import pandas as pd
import datetime as dt

In [544]:
pesquisa_perguntas = pd.read_csv('./data/disciplinas/forms/resposta_pesq.csv', delimiter=';')
pesquisa_perguntas['timestamp'] = list(map(lambda x: dt.datetime.strptime(x, '%m/%d/%Y %H:%M:%S'), pesquisa_perguntas['Carimbo de data/hora']))
pesquisa_perguntas = pesquisa_perguntas[pesquisa_perguntas['timestamp'] >= dt.datetime(2022, 10, 7, 0, 0, 0)]
len(pesquisa_perguntas)

4395

In [545]:
pesquisa = pd.DataFrame()
pesquisa['vinculo'] = pesquisa_perguntas['Qual o seu vínculo com a USP?']
pesquisa['instituto'] = pesquisa_perguntas['A qual instituto você está associado?']


# ida
pesquisa['utiliza_circular_ida'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para entrar na USP?'] >= 2
pesquisa['linhas_ida'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?']
pesquisa['ponto_embarque_ida'] = pesquisa_perguntas['Por qual ponto você normalmente embarca?']
pesquisa['ponto_desembarque_ida'] = pesquisa_perguntas['Em qual ponto de ônibus você normalmente desembarca?']
pesquisa['horario_ida'] = pesquisa_perguntas['Que horas você normalmente chega na USP?']


# volta
pesquisa['utiliza_circular_volta'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para sair da USP?'] >= 2
pesquisa['linhas_volta'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?.1']
pesquisa['ponto_embarque_volta'] = pesquisa_perguntas['Em qual o ponto de ônibus você embarca para sair da USP?']
pesquisa['ponto_desembarque_volta'] = pesquisa_perguntas['Por qual ponto você normalmente desembarca?']
pesquisa['horario_volta'] = pesquisa_perguntas['Que horas você normalmente chega no ponto de ônibus para sair da USP?']

pesquisa.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


##### Filtrar estudantes

In [546]:
pesquisa[['vinculo']].value_counts()

vinculo                                                        
Aluno de graduação                                                 2880
Aluno de pós-graduação                                              752
Funcionário                                                         644
Docente                                                              36
docente                                                              22
Professor                                                            21
professor                                                             8
Ex-aluno                                                              5
Professor                                                             5
Docente                                                               4
Professor titular                                                     1
aprimoramento                                                         1
Pós-doutorando                                                        1


In [547]:
pesquisa[['vinculo', 'utiliza_circular_ida']].value_counts()

vinculo                                                          utiliza_circular_ida
Aluno de graduação                                               True                    2532
Aluno de pós-graduação                                           True                     575
Funcionário                                                      True                     369
Aluno de graduação                                               False                    348
Funcionário                                                      False                    275
Aluno de pós-graduação                                           False                    177
Docente                                                          True                      28
docente                                                          False                     17
Professor                                                        True                      11
                                                                 Fal

In [548]:
def grad_ou_pos(x):
    return x == 'Aluno de graduação' or x == 'Aluno de pós-graduação'
pesquisa_alunos = pesquisa[list(map(grad_ou_pos, pesquisa['vinculo']))]
pesquisa_alunos[['vinculo']].value_counts()

vinculo               
Aluno de graduação        2880
Aluno de pós-graduação     752
dtype: int64

##### Filtrar respostas pesquisa pelo Intervalo Horário

In [549]:
pesquisa_alunos.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


In [550]:
pesquisa_ida = pesquisa_alunos[pesquisa_alunos['horario_ida'].notna()]
pesquisa_alunos_intervalo = pesquisa_ida[list(map(intervalo_horario_am_pm, pesquisa_ida['horario_ida']))]
pesquisa_alunos_intervalo

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
141,Aluno de graduação,Faculdade de Ciências Farmacêuticas (FCF),True,8022,P3,Farmácia e Química,6:55:00 PM,True,8022,Butantan,Estação Butantã,10:00:00 PM
144,Aluno de graduação,"Faculdade de Economia, Administração e Contabi...",True,"8012, 8032",Estação Butantã,FEA,6:15:00 PM,True,8032,FEA,Estação Butantã,10:00:00 PM
153,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022, 8032",Estação Butantã,FEA,5:40:00 PM,True,"8022, 8032",FEA,Estação Butantã,9:00:00 PM
154,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,"8012, 8032",Estação Butantã,Biblioteca Brasiliana,7:00:00 PM,True,"8022, 8032",Biblioteca Brasiliana,Estação Butantã,10:40:00 PM
155,Aluno de graduação,Instituto de Física (IF),True,"8012, 8022","R. Prof. Mello Moraes, 1473 (localizado entre ...",Física,6:40:00 PM,True,8012,Prefeitura/Física,P3,10:50:00 PM
...,...,...,...,...,...,...,...,...,...,...,...,...
4498,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8022, 8032",Estação Butantã,FEA,7:00:00 PM,True,"8012, 8022, 8032",FAU I (R do Matão),Estação Butantã,10:30:00 PM
4506,Aluno de graduação,Escola de Comunicações e Artes (ECA),True,"8012, 8022",Estação Butantã,ECA,7:00:00 PM,True,"8012, 8032",ECA,Estação Butantã,10:30:00 PM
4507,Aluno de graduação,Faculdade de Educação (FE),True,8012,Estação Butantã,P1,6:15:00 PM,True,8012,P1,Estação Butantã,10:00:00 PM
4519,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8022,PTREM,Letras,6:00:00 PM,True,8032,Biblioteca Brasiliana,PTREM,10:10:00 PM


##### Normaliza instituto com unidade do matrusp

In [551]:
def retira_sigla(x):
    index = x.find('(')
    
    if index < 0:
        return x
    
    return x[:index - 1]

def filtra_instituto_pesq(x):
    x = retira_sigla(x)
    
    if x in ['ECA e MAC USP', 'Escola de Artes, Ciências e Humanidades']: 
        return 'Escola de Comunicações e Artes'
    elif x == 'FFLCH':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x in ['IPEN', 'IPEN ', 'ipen', 'Instituto de pesquisas energéticas e nucleares', 'Instituto de Pesquisa Energéticas e Nucleares', 'Instituto de Pesquisas Energéticas e Nucleares - IPEN', 'Instituto de pesquisas energéticas e nucleares ', 'Instituto de pesquisas energéticas e nucleares -IPEN', 'Instituo de Pesquisas Energéticas e Nucleares']:
        return 'Instituto de Pesquisas Energéticas e Nucleares'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    elif x in ['PRG', 'Prg ', 'Pró-Reitoria de Graduação', 'Pró-Reitoria de graduação', 'Pró-reitoria de graduacao', 'Curso de Ciências Moleculares']:
        return None
    elif x == 'Faculadae de Arquitetura e Urbanismo':
        return 'Faculdade de Arquitetura e Urbanismo'
    elif x == 'Instituo de Astronomia, Geofísica e Ciências Atmosféricas':
        return 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas'
    elif x == 'Faculdade de Economia, Administração e Contabilidade':
        return 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    elif x in ['Escola de Aplicação da Faculdade de Educação (EA-FEUSP)']:
        return 'Escola de Educação Física e Esporte'
    elif x == 'Instituto de Relações internacionais':
        return 'Instituto de Relações Internacionais'
    else:
        return x
    
pesquisa_alunos_intervalo2 = pesquisa_alunos_intervalo.copy()
pesquisa_alunos_intervalo2['instituto'] = list(map(filtra_instituto_pesq, pesquisa_alunos_intervalo['instituto']))

In [552]:
# Filtra os Nones

pesquisa_alunos_intervalo = pesquisa_alunos_intervalo2
pesquisa_alunos_intervalo = pesquisa_alunos_intervalo[pesquisa_alunos_intervalo['instituto'].notna()]
pesquisa_alunos_intervalo['instituto']

141                   Faculdade de Ciências Farmacêuticas
144     Faculdade de Economia, Administração, Contabil...
153                 Instituto de Matemática e Estatística
154     Faculdade de Filosofia, Letras e Ciências Humanas
155                                   Instituto de Física
                              ...                        
4498                 Faculdade de Arquitetura e Urbanismo
4506                       Escola de Comunicações e Artes
4507                                Faculdade de Educação
4519    Faculdade de Filosofia, Letras e Ciências Humanas
4526                             Instituto de Biociências
Name: instituto, Length: 496, dtype: object

##### Mapeia Institutos Fora CUASO

In [553]:
def mapeia_cursos_de_fora(x):
    if x in ['Faculdade de Medicina', 'Escola de Enfermagem', 'Faculdade de Saúde Pública']:
        return 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    elif x == 'Faculdade de Direito':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    else:
        return x
    
pesquisa_alunos_intervalo2 = pesquisa_alunos_intervalo.copy()
pesquisa_alunos_intervalo2['instituto'] = list(map(mapeia_cursos_de_fora, pesquisa_alunos_intervalo['instituto']))
pesquisa_alunos_intervalo = pesquisa_alunos_intervalo2

In [554]:
pesquisa_alunos_intervalo['instituto'].unique()

array(['Faculdade de Ciências Farmacêuticas',
       'Faculdade de Economia, Administração, Contabilidade e Atuária',
       'Instituto de Matemática e Estatística',
       'Faculdade de Filosofia, Letras e Ciências Humanas',
       'Instituto de Física', 'Escola Politécnica',
       'Faculdade de Arquitetura e Urbanismo', 'Faculdade de Odontologia',
       'Instituto de Química', 'Escola de Comunicações e Artes',
       'Faculdade de Educação', 'Instituto de Biociências',
       'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
       'Instituto de Relações Internacionais',
       'Faculdade de Medicina Veterinária e Zootecnia',
       'Escola de Educação Física e Esporte',
       'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
       'Instituto de Pesquisas Energéticas e Nucleares',
       'Instituto de Geociências', 'Ipen', 'Instituto Oceanográfico',
       'Instituto de Estudos Brasileiros',
       'Instituto d

##### Filtrar Butantã

In [555]:
def is_butanta(x): 
    return x == 'Estação Butantã' or x == 'PTREM'

pesquisa_alunos_intervalo_butanta = pesquisa_alunos_intervalo[pesquisa_alunos_intervalo['utiliza_circular_ida']]
pesquisa_alunos_intervalo_butanta = pesquisa_alunos_intervalo_butanta[list(map(is_butanta, pesquisa_alunos_intervalo_butanta['ponto_embarque_ida']))]
pesquisa_alunos_intervalo_butanta

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
144,Aluno de graduação,"Faculdade de Economia, Administração, Contabil...",True,"8012, 8032",Estação Butantã,FEA,6:15:00 PM,True,8032,FEA,Estação Butantã,10:00:00 PM
153,Aluno de graduação,Instituto de Matemática e Estatística,True,"8012, 8022, 8032",Estação Butantã,FEA,5:40:00 PM,True,"8022, 8032",FEA,Estação Butantã,9:00:00 PM
154,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Humanas",True,"8012, 8032",Estação Butantã,Biblioteca Brasiliana,7:00:00 PM,True,"8022, 8032",Biblioteca Brasiliana,Estação Butantã,10:40:00 PM
166,Aluno de graduação,Escola Politécnica,True,"8012, 8032",Estação Butantã,Poli Civil,7:00:00 AM,True,"8012, 8032",Poli Civil,Estação Butantã,3:00:00 PM
168,Aluno de graduação,Escola Politécnica,True,8022,PTREM,Poli Civil,6:50:00 AM,True,8012,Poli Civil,PTREM,2:50:00 PM
...,...,...,...,...,...,...,...,...,...,...,...,...
4498,Aluno de graduação,Faculdade de Arquitetura e Urbanismo,True,"8012, 8022, 8032",Estação Butantã,FEA,7:00:00 PM,True,"8012, 8022, 8032",FAU I (R do Matão),Estação Butantã,10:30:00 PM
4506,Aluno de graduação,Escola de Comunicações e Artes,True,"8012, 8022",Estação Butantã,ECA,7:00:00 PM,True,"8012, 8032",ECA,Estação Butantã,10:30:00 PM
4507,Aluno de graduação,Faculdade de Educação,True,8012,Estação Butantã,P1,6:15:00 PM,True,8012,P1,Estação Butantã,10:00:00 PM
4519,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Humanas",True,8022,PTREM,Letras,6:00:00 PM,True,8032,Biblioteca Brasiliana,PTREM,10:10:00 PM


##### Mapeia Pontos Pesquisa para ficar igual ao da API

In [556]:
import requests
pontos = requests.get('https://uspdigital.usp.br/mobile/servicos/mapa/locais?campus=cidade-universitaria&categorias=transportes')
pontos = json.loads(pontos.text)
pontos[10]

{'categoria': 'transportes',
 'campus': 'cidade-universitaria',
 'id': '1811',
 'titulo': 'Ponto de ônibus ECA',
 'latitude': '-23.557806338669685',
 'longitude': '-46.72689100000002',
 'endereco': 'Av. Prof. Lúcio Martins Rodrigues',
 'imagem': 'http://www.usp.br/mapas/wp-content/uploads/10042014giroculturalfotomarcossantos008.jpg',
 'url': '',
 'descricao': 'Atende as linhas  701U-10, 702U-10, 7725-10, 8012-10'}

In [557]:
nomes_pontos_req = []
for ponto in pontos:
    if ponto['campus'] == 'cidade-universitaria':
        nomes_pontos_req.append(ponto['titulo'])
 
nomes_pontos_pesq = []
for ponto in pesquisa_alunos_intervalo['ponto_desembarque_ida'].unique():
    nomes_pontos_pesq.append(ponto)

print(nomes_pontos_req)
print(nomes_pontos_pesq)

['Ponto de ônibus metrô Butantã', 'Ponto de ônibus Afrânio Peixoto', 'Ponto de ônibus Poli Metalúrgica', 'Terminal de ônibus USP', 'Ponto de ônibus Poli Mecânica', 'Ponto de ônibus portaria II', 'Ponto de ônibus Hidráulica', 'Ponto de Psicologia I', 'Ponto de ônibus Barracões', 'Ponto de ônibus ECA', 'Ponto de ônibus Praça do Relógio', 'Ponto de ônibus - Praça do Relógio', 'Ponto de ônibus Psicologia II', 'Ponto de ônibus - Acesso CPTM II', 'Ponto de ônibus - Acesso CPTM I', 'Ponto de ônibus Escola de Educação Física II', 'Ponto de ônibus Escola de Educação Física I', 'Ponto de ônibus Academia de Polícia', 'Ponto de ônibus Paços das Artes', 'Ponto de ônibus Educação', 'Ponto de ônibus CRUSP', 'Ponto de ônibus Cultura Japonesa', 'Ponto de ônibus Biblioteca Brasiliana', 'Ponto de ônibus Letras', 'Ponto de ônibus Geociência', 'Ponto de ônibus Reitoria/Bancos', 'Ponto de ônibus FEA', 'Ponto de ônibus FAU II', 'Ponto de ônibus Poli Biênio', 'Ponto de ônibus Poli Eletrotécnica', 'Ponto de ôn

In [558]:
def iguais_com_prefixo(x):
    aux_nomes_pontos_pesq = ['Ponto de ônibus ' + x for x in nomes_pontos_pesq]
    if 'Ponto de ônibus ' + x in [item for item in aux_nomes_pontos_pesq if item in list(set(nomes_pontos_req) & set(aux_nomes_pontos_pesq))]:
        return 'Ponto de ônibus ' + x
    else:
        return x
 
def pesquisa_to_pontos_req(x):
    if x in ["Geografia/História", "História e Geografia", "Químicas"]:
        return "Ponto e ônibus História e Geografia"
    elif x  == "Poli Metalurgia":
        return "Ponto de ônibus Poli Metalúrgica"
    elif x == "P3":
        return "Ponto de ônibus Portaria III"
    elif x == "Farmácia e Química":
        return "Ponto de ônibus Biblioteca Farmácia e Química"
    elif x in ["CRUSP (Praça da Reitoria)", "R. Prof. Mello Moraes, 1473 (localizado entre a CRUSP e a Raia, sentindo Metrô Butantã)", "R. Prof. Mello Moraes, 900 (localizado entre a CRUSP e a Raia, sentindo P3)"]:
        return "Ponto de ônibus CRUSP"
    elif x == "Biomédicas":
        return "Ponto de ônibus Biomédicas III"
    elif x == "Prefeitura/Física":
        return "Ponto de ônibus Ponto Clube dos funcionários"
    elif x == "Educação Física":
        return "Ponto de ônibus Escola de Educação Física I"
    elif x in ["Estação Butantã", "Terminal Cidade Universitária"]:
        return "Ponto de ônibus metrô Butantã"
    elif x == "Poli-Eletrotécnia":
        return "Ponto de ônibus Poli Eletrotécnica"
    elif x == "Geociências":
        return "Ponto de ônibus Geociências I"
    elif x in ["icb ", "Biociências", "ICB 1 (Av. Prof. Lineu Prestes, 1524)"]:
        return "Ponto de ônibus  Parada Matemática"
    elif x in ["FE", "Avenida da Universidade", "Faculdade de Educação ", "Educação / CEPEUSP", " FE", "Prédio da FE", "Educação ", "Educação", "Faculdade de Educação"]:
        return "Ponto de ônibus Educação"
    elif x in ["Rua do matão frente da FAU", "IO", "Rua do matao, 0", "Rua do Matão 0 (FAU)", "Rua do Matão (na frente da FAU)", "Intuito Oceanografico", "R. do Matão", "Oceanografia ", "Oceanógrafico/Matão", "Rua do Matão, 0", "R. Do matão ",  "Rua do Matão", "Ponto em frente à FAU na Rua do Matão"]:
        return "Ponto de ônibus FAU I"
    elif x in ["depois da biblioteca da química ", "Rua do Lago", "Lineu Prestes entre a biblioteca da quimica e o icb", "Ponto do Lago", "Av. Prof. Lineu Prestes, 0"]:
        return "Ponto de ônibus rua do Lago"
    elif x == "Psicologia I":
        return "Ponto de ônibus Psicologia II"
    elif x == "Cepam":
        return "Ponto de ônibus CEPAM"
    elif x in ["Ipen", "FO ou Ipen", "Marinha/ Odontologia"]:
        return "Ponto de ônibus IPEN"
    elif x in ["Acesso Vila Indiana", "Portão da Vila Indiana"]:
        return "Ponto de ônibus acesso Vl. Indiana"
    elif x == "PTREM":
        return "Ponto de ônibus - Acesso CPTM I"
    elif x == "IPT":
        return "Ponto de ônibus Cocesp I"
    elif x in ["Centro De Difusão Internacional", "Poli Hidráulica"]:
        return "Ponto de ônibus Hidráulica"
    elif x == "Avenida Professor Luciano Gualberto (depois da Parada I.P.T, no 8022)":
        return "Ponto de ônibus Poli Eletrotécnica"
    elif x in ["P1", "Instituto Butantan "]:
        return "Ponto de ônibus Academia de Polícia"
    elif x == "P2":
        return "Ponto de ônibus portaria II"
    elif x in ["FAU ", "FEA ou FAU II", "As vezes desembarco na FAU II ou na FEA"]:
        return "Ponto de ônibus FAU II"
    elif x == "Portaria São Remo":
        return "Ponto de ônibus acesso Rio Pequeno"
    elif x == "If e Ipen":
        return "Ponto de ônibus Física"
    elif x == "Casa de Cultura Japonesa, ETEC CEPAM":
        return "Ponto de ônibus Cultura Japonesa"
    elif x == "IEB":
        return "Ponto de ônibus Biblioteca Brasiliana"
    elif x == "IAG (ou, às vezes, Física)":
        return "Ponto de ônibus IAG"
    elif x == "Prof Almeida Prado":
        return "Terminal de ônibus USP"
    elif x == "Depende do dia":
        return None
    else:
        return x

In [559]:
pesquisa_alunos_intervalo_butanta['ponto_desembarque_ida'] = list(map(lambda x: pesquisa_to_pontos_req(iguais_com_prefixo(x)), pesquisa_alunos_intervalo_butanta['ponto_desembarque_ida']))
pesquisa_alunos_intervalo_butanta = pesquisa_alunos_intervalo_butanta[pesquisa_alunos_intervalo_butanta['ponto_desembarque_ida'].notna()]
pesquisa_alunos_intervalo_butanta['ponto_desembarque_ida'].unique()

array(['Ponto de ônibus FEA', 'Ponto de ônibus Biblioteca Brasiliana',
       'Ponto de ônibus Poli Civil', 'Ponto de ônibus Odontologia',
       'Ponto de ônibus CRUSP',
       'Ponto de ônibus Biblioteca Farmácia e Química',
       'Ponto de ônibus Ponto Clube dos funcionários',
       'Ponto de ônibus Poli Biênio', 'Ponto de ônibus Reitoria/Bancos',
       'Ponto e ônibus História e Geografia', 'Ponto de ônibus CEPAM',
       'Ponto de ônibus Educação', 'Ponto de ônibus Letras',
       'Ponto de ônibus  Parada Matemática',
       'Ponto de ônibus Hospital Universitário',
       'Ponto de ônibus metrô Butantã',
       'Ponto de ônibus Praça do Relógio', 'Ponto de ônibus FAU II',
       'Ponto de ônibus Poli Eletrotécnica',
       'Ponto de ônibus Academia de Polícia',
       'Ponto de ônibus acesso Vl. Indiana',
       'Ponto de ônibus Biomédicas III', 'Ponto de ônibus ECA',
       'Ponto de ônibus Física', 'Ponto de ônibus IPEN',
       'Ponto de ônibus Geociências I',
       'No pr

##### Calcular Porcentagem Carro por Instituto

In [560]:
porc_circular_por_instituto = pesquisa_alunos_intervalo_butanta[['instituto','utiliza_circular_ida']].groupby('instituto').agg('mean').reset_index()
porc_circular_por_instituto

,instituto,utiliza_circular_ida
0,Escola Politécnica,1.0
1,Escola de Comunicações e Artes,1.0
2,Escola de Educação Física e Esporte,1.0
3,Faculdade de Arquitetura e Urbanismo,1.0
4,Faculdade de Ciências Farmacêuticas,1.0
5,"Faculdade de Economia, Administração, Contabil...",1.0
6,Faculdade de Educação,1.0
7,"Faculdade de Filosofia, Letras e Ciências Humanas",1.0
8,"Faculdade de Medicina, Instituto de Ciências B...",1.0
9,Faculdade de Odontologia,1.0


In [561]:
def porc_utiliza_circular(instituto):
    return porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_ida'].iloc[0]
porc_utiliza_circular('Instituto de Física')

1.0

##### Calcula porcentagem para Ponto e Linha por Instituto (filtrando quem usa carro)

In [562]:
import random
def usa_linha(str_linhas, linha_target):
    linhas = str_linhas.split(', ')
    linha_selecionada = linhas[random.randint(0, len(linhas) - 1)]
    return (linha_selecionada == linha_target)

def calcula_quantidade_pessoas_ponto(instituto, ponto, linha):
    df_instituto = pesquisa_alunos_intervalo_butanta[pesquisa_alunos_intervalo_butanta['instituto'] == instituto]

    df_linha = df_instituto[list(map(lambda x: usa_linha(x, linha), df_instituto['linhas_ida']))]    
    if len(df_linha['linhas_ida']) == 0:
        return 0

    df_ponto = df_linha[df_linha['ponto_desembarque_ida'] == ponto]
    if len(df_ponto['ponto_desembarque_ida']) == 0:
        return 0

    return df_ponto['ponto_desembarque_ida'].value_counts().iloc[0]

In [563]:
pontos_existentes = pesquisa_alunos_intervalo_butanta['ponto_desembarque_ida'].unique()

institutos_ponto_rel = {}
for instituto in pesquisa_alunos_intervalo_butanta['instituto'].unique():
    institutos_ponto_rel[instituto] = {}
    soma = len(pesquisa_alunos_intervalo_butanta[pesquisa_alunos_intervalo_butanta['instituto'] == instituto]['instituto'])
    for ponto in pontos_existentes:
        for linha in ['8012', '8022', '8032']:
            pessoas = calcula_quantidade_pessoas_ponto(instituto, ponto, linha) * porc_utiliza_circular(instituto)
            if pessoas > 0:
                if ponto not in institutos_ponto_rel[instituto]:
                    institutos_ponto_rel[instituto][ponto] = {}
                institutos_ponto_rel[instituto][ponto][linha] = pessoas / soma
institutos_ponto_rel

{'Faculdade de Economia, Administração, Contabilidade e Atuária': {'Ponto de ônibus FEA': {'8012': 0.2631578947368421,
   '8022': 0.15789473684210525,
   '8032': 0.3684210526315789},
  'Ponto de ônibus CRUSP': {'8012': 0.02631578947368421},
  'Ponto de ônibus Ponto Clube dos funcionários': {'8012': 0.02631578947368421,
   '8032': 0.02631578947368421},
  'Ponto de ônibus Poli Biênio': {'8012': 0.02631578947368421}},
 'Instituto de Matemática e Estatística': {'Ponto de ônibus FEA': {'8012': 0.23809523809523808,
   '8022': 0.047619047619047616,
   '8032': 0.19047619047619047},
  'Ponto de ônibus CRUSP': {'8012': 0.047619047619047616},
  'Ponto de ônibus metrô Butantã': {'8012': 0.047619047619047616},
  'Ponto de ônibus Praça do Relógio': {'8012': 0.047619047619047616},
  'Ponto de ônibus FAU II': {'8012': 0.047619047619047616,
   '8022': 0.09523809523809523,
   '8032': 0.047619047619047616},
  'Ponto de ônibus Poli Eletrotécnica': {'8012': 0.047619047619047616,
   '8022': 0.04761904761904

# Pesquisa e Matrusp

##### Verificar unidades Matrusp e Pesquisa

In [564]:
# Alterar a unidade de aulas na qual a unidade não respondeu a pesquisa

for aula in aulas_intervalo_horario:
    if aula['unidade'] == 'Instituto de Ciências Biomédicas':
        aula['unidade'] = 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    if aula['unidade'] == 'Instituto de Estudos Brasileiros':
        aula['unidade'] = 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    if aula['unidade'] == 'Instituto de Psicologia':
        aula['unidade'] = 'Escola de Comunicações e Artes'

In [565]:
institutos_matrusp = {}
for aula in aulas_intervalo_horario:
    institutos_matrusp[aula['unidade']] = 1
institutos_matrusp = list(institutos_matrusp.keys())
institutos_matrusp.sort()
institutos_matrusp

['Escola Politécnica',
 'Escola de Comunicações e Artes',
 'Faculdade de Arquitetura e Urbanismo',
 'Faculdade de Ciências Farmacêuticas',
 'Faculdade de Economia, Administração, Contabilidade e Atuária',
 'Faculdade de Educação',
 'Faculdade de Filosofia, Letras e Ciências Humanas',
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
 'Faculdade de Odontologia',
 'Instituto Oceanográfico',
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
 'Instituto de Biociências',
 'Instituto de Física',
 'Instituto de Geociências',
 'Instituto de Matemática e Estatística',
 'Instituto de Química',
 'Instituto de Relações Internacionais']

In [566]:
institutos_pesquisa = list(institutos_ponto_rel.keys())
institutos_pesquisa.sort()
institutos_pesquisa

['Escola Politécnica',
 'Escola de Comunicações e Artes',
 'Escola de Educação Física e Esporte',
 'Faculdade de Arquitetura e Urbanismo',
 'Faculdade de Ciências Farmacêuticas',
 'Faculdade de Economia, Administração, Contabilidade e Atuária',
 'Faculdade de Educação',
 'Faculdade de Filosofia, Letras e Ciências Humanas',
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
 'Faculdade de Odontologia',
 'Instituto Oceanográfico',
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
 'Instituto de Biociências',
 'Instituto de Física',
 'Instituto de Geociências',
 'Instituto de Matemática e Estatística',
 'Instituto de Pesquisas Energéticas e Nucleares',
 'Instituto de Química',
 'Instituto de Relações Internacionais']

In [567]:
error = False
for instituto_matrusp in institutos_matrusp:
    if instituto_matrusp not in institutos_pesquisa:
        print(instituto_matrusp + ' from matrusp is not in institutos pesquisa')
        error = True
if error:
    raise Exception('error')

##### Calcula Map: dia->hora->ponto->linha->pessoas

In [568]:
dia_hora_ponto_linha = {}

def sum_valor_dia_hora_ponto_linha(dia, hora, ponto, linha, valor):
    if dia not in dia_hora_ponto_linha:
        dia_hora_ponto_linha[dia] = {}
    if hora not in dia_hora_ponto_linha[dia]:
        dia_hora_ponto_linha[dia][hora] = {}
    if ponto not in dia_hora_ponto_linha[dia][hora]:
        dia_hora_ponto_linha[dia][hora][ponto] = {}
    if linha not in dia_hora_ponto_linha[dia][hora][ponto]:
        dia_hora_ponto_linha[dia][hora][ponto][linha] = 0
    dia_hora_ponto_linha[dia][hora][ponto][linha] += valor

for aula in aulas_intervalo_horario:
    porcentagem_pontos_por_instituto = institutos_ponto_rel[aula['unidade']]
    for ponto in porcentagem_pontos_por_instituto:
        linhas = porcentagem_pontos_por_instituto[ponto]
        for linha in linhas:
            porc = linhas[linha]
            valor = aula['total_matriculados'] * porc
            sum_valor_dia_hora_ponto_linha(aula['dia'], hora, ponto, linha, valor)

# arredondando
for dia in dia_hora_ponto_linha:
    for hora in dia_hora_ponto_linha[dia]:
        for ponto in dia_hora_ponto_linha[dia][hora]:
            for linha in dia_hora_ponto_linha[dia][hora][ponto]:
                dia_hora_ponto_linha[dia][hora][ponto][linha] = round(dia_hora_ponto_linha[dia][hora][ponto][linha])

dia_hora_ponto_linha

{'qua': {1080: {'Ponto de ônibus Reitoria/Bancos': {'8012': 38, '8032': 19},
   'Ponto de ônibus Geociências I': {'8022': 19, '8032': 19},
   'Ponto de ônibus Biblioteca Farmácia e Química': {'8022': 318,
    '8012': 54,
    '8032': 8},
   'Ponto de ônibus  Parada Matemática': {'8012': 74, '8022': 118, '8032': 18},
   'Ponto de ônibus acesso Vl. Indiana': {'8022': 21, '8012': 2},
   'Ponto de ônibus Biomédicas III': {'8022': 26, '8012': 8},
   'Ponto de ônibus rua do Lago': {'8012': 18, '8022': 37},
   'Ponto de ônibus - Acesso CPTM I': {'8022': 18},
   'Rua do lago': {'8022': 18},
   'Ponto de ônibus FEA': {'8012': 77, '8022': 20, '8032': 79},
   'Ponto de ônibus CRUSP': {'8012': 25, '8022': 35, '8032': 1},
   'Ponto de ônibus metrô Butantã': {'8012': 33, '8022': 10},
   'Ponto de ônibus Praça do Relógio': {'8012': 9},
   'Ponto de ônibus FAU II': {'8012': 20, '8022': 61, '8032': 20},
   'Ponto de ônibus Poli Eletrotécnica': {'8012': 9, '8022': 16},
   'Ponto de ônibus Biblioteca Bras

##### Salva Map em Arquivo

In [569]:
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def encode(s):
    return s.encode('latin1').decode('unicode_escape')

with open(map_file_name, 'w+') as f:
    str_dia_hora_ponto_linha = encode(json.dumps(dia_hora_ponto_linha, indent=4, cls=NpEncoder))
    f.write(str_dia_hora_ponto_linha)
    print(str_dia_hora_ponto_linha)

{
    "qua": {
        "1080": {
            "Ponto de ônibus Reitoria/Bancos": {
                "8012": 38,
                "8032": 19
            },
            "Ponto de ônibus Geociências I": {
                "8022": 19,
                "8032": 19
            },
            "Ponto de ônibus Biblioteca Farmácia e Química": {
                "8022": 318,
                "8012": 54,
                "8032": 8
            },
            "Ponto de ônibus  Parada Matemática": {
                "8012": 74,
                "8022": 118,
                "8032": 18
            },
            "Ponto de ônibus acesso Vl. Indiana": {
                "8022": 21,
                "8012": 2
            },
            "Ponto de ônibus Biomédicas III": {
                "8022": 26,
                "8012": 8
            },
            "Ponto de ônibus rua do Lago": {
                "8012": 18,
                "8022": 37
            },
            "Ponto de ônibus - Acesso CPTM I": {
                "

## Add employees and professors
TODO: considerar funcionários e professores estrapolando o horário de entrada da pesquisa

## Outros

##### Número Pessoas por Dia e Instituto

In [570]:
agrupamento_dia_instituto = {}
agrupamento_dia_instituto['seg'] = {}
agrupamento_dia_instituto['ter'] = {}
agrupamento_dia_instituto['qua'] = {}
agrupamento_dia_instituto['qui'] = {}
agrupamento_dia_instituto['sex'] = {}
agrupamento_dia_instituto['sab'] = {}
agrupamento_dia_instituto['dom'] = {}
for aula in aulas_intervalo_horario:
    if aula['unidade'] not in agrupamento_dia_instituto[aula['dia']]:
        agrupamento_dia_instituto[aula['dia']][aula['unidade']] = 0
    agrupamento_dia_instituto[aula['dia']][aula['unidade']] += aula['total_matriculados']
agrupamento_dia_instituto

{'seg': {'Instituto de Biociências': 164,
  'Instituto de Física': 367,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 79,
  'Faculdade de Odontologia': 310,
  'Instituto de Matemática e Estatística': 60,
  'Faculdade de Ciências Farmacêuticas': 235,
  'Instituto de Química': 220,
  'Faculdade de Arquitetura e Urbanismo': 137,
  'Instituto de Geociências': 49,
  'Escola Politécnica': 93,
  'Escola de Comunicações e Artes': 41,
  'Instituto Oceanográfico': 25,
  'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 291,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 28},
 'ter': {'Instituto de Matemática e Estatística': 152,
  'Instituto de Física': 278,
  'Instituto de Biociências': 252,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 69,
  'Faculdade de Odontologia': 474,
  'Faculdade de Ciências Farmacêuticas': 178,
  'Instituto de Química': 220,
  'Faculdade de Arquitetura e Urbanismo': 72,
  'Inst

In [571]:
for dia in agrupamento_dia_instituto:
    for instituto in agrupamento_dia_instituto[dia]:
        agrupamento_dia_instituto[dia][instituto] *= porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_ida'].iloc[0]
        agrupamento_dia_instituto[dia][instituto] = int(agrupamento_dia_instituto[dia][instituto])

# Pessoas por instituto que utilizam circular de acordo com a pesquisa
agrupamento_dia_instituto

{'seg': {'Instituto de Biociências': 164,
  'Instituto de Física': 367,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 79,
  'Faculdade de Odontologia': 310,
  'Instituto de Matemática e Estatística': 60,
  'Faculdade de Ciências Farmacêuticas': 235,
  'Instituto de Química': 220,
  'Faculdade de Arquitetura e Urbanismo': 137,
  'Instituto de Geociências': 49,
  'Escola Politécnica': 93,
  'Escola de Comunicações e Artes': 41,
  'Instituto Oceanográfico': 25,
  'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 291,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 28},
 'ter': {'Instituto de Matemática e Estatística': 152,
  'Instituto de Física': 278,
  'Instituto de Biociências': 252,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 69,
  'Faculdade de Odontologia': 474,
  'Faculdade de Ciências Farmacêuticas': 178,
  'Instituto de Química': 220,
  'Faculdade de Arquitetura e Urbanismo': 72,
  'Inst